# Testing KLU versus SparseLU in Fault Simulation 

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import dpsimpy
import glob
import requests

# %matplotlib widget

## Configure Simulation Scenario

In [ ]:
example_name = "DP_WSCC9bus_SGReducedOrderVBR"

final_time = 2.0
time_step = 100e-6
with_fault = True
start_time_fault = 1.2
end_time_fault = 1.3

sg_type = "4"
fault_bus_name = "BUS6"
log_dir = "logs"

log_level = dpsimpy.LogLevel.trace

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

filename = 'WSCC-09_Dyn_Fourth'

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_Dyn_Fourth/WSCC-09_Dyn_Fourth'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')
    
files = glob.glob(filename+'_*.xml')

## Run Powerflow Simulation

In [ ]:
sim_name_pf = example_name + "_PF"

dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, log_level, log_level)

system_pf = reader.loadCIM(60, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system_pf.component('GEN1').modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)
logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + ".V", node.attr("v"))
    
sim_pf = dpsimpy.Simulation(sim_name_pf, log_level)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(final_time)
sim_pf.set_final_time(2*final_time)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)
sim_pf.add_logger(logger_pf)
sim_pf.run()

## Define Dynamic Simulation Function

In [ ]:
def run_simulation(sim_name, implementation, model_syngens_as_norton, config = dpsimpy.DirectLinearSolverConfiguration()):
   
    dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name)
    reader2 = dpsimpy.CIMReader(sim_name, log_level, log_level)
    if sg_type == "3":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG3OrderVBR)
    elif sg_type == "4":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG4OrderVBR)
    elif sg_type == "6b":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG6bOrderVBR)
    else:
        raise Exception("Unsupported reduced-order SG type!")

    fault_dp = dpsimpy.sp.ph1.Switch("Fault", log_level)

    if with_fault == True:
        n1_dp = dpsimpy.dp.SimNode(fault_bus_name, dpsimpy.PhaseType.Single)
        fault_dp.set_parameters(1e12, 0.02*529)
        fault_dp.connect([dpsimpy.dp.SimNode.gnd, n1_dp])
        fault_dp.open()
        sys.add(fault_dp)

    sys.init_with_powerflow(system_pf, dpsimpy.Domain.DP)
    
    for comp in sys.components:
       if comp.__class__ == dpsimpy.dp.ph1.SynchronGenerator4OrderVBR:
           gen_pf = system_pf.component(comp.name())
           comp.get_terminal(index=0).set_power(- gen_pf.get_apparent_power())
           comp.set_model_as_norton_source(model_syngens_as_norton)

    logger = dpsimpy.Logger(sim_name)
    for node in sys.nodes:
        logger.log_attribute(node.name() + ".V", node.attr("v"))

    for comp in sys.components:
        if comp.__class__ == dpsimpy.dp.ph1.SynchronGenerator4OrderVBR:
            logger.log_attribute(comp.name() + ".Tm", comp.attr("Tm"))
            logger.log_attribute(comp.name() + ".Te", comp.attr("Te"))
            logger.log_attribute(comp.name() + ".omega", comp.attr("w_r"))
            logger.log_attribute(comp.name() + ".delta", comp.attr("delta"))

    sim = dpsimpy.Simulation(sim_name, log_level)
    sim.set_system(sys)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.do_system_matrix_recomputation(True)
    sim.set_direct_solver_implementation(implementation)
    sim.set_direct_linear_solver_configuration(config)
    sim.add_logger(logger)

    if with_fault:
        fault_event1 = dpsimpy.event.SwitchEvent(start_time_fault, fault_dp, True)
        fault_event2 = dpsimpy.event.SwitchEvent(end_time_fault, fault_dp, False)
        sim.add_event(fault_event1)
        sim.add_event(fault_event2)

    sim.run()

## Run Dynamic Simulation with SparseLU

- thevenin

In [ ]:
%%capture
sim_name_lu_thevenin = example_name+'_SparseLU_thevenin'
run_simulation(sim_name_lu_thevenin,dpsimpy.DirectLinearSolverImpl.SparseLU,False)
ts_dpsim_sparse_thevenin = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_thevenin+".csv")
ts_dpsim_sparse_thevenin_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_thevenin+"_RightVector.csv")
ts_dpsim_sparse_thevenin_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_thevenin+"_LeftVector.csv")

- norton

In [ ]:
%%capture
sim_name_lu_norton = example_name+'_SparseLU_norton'
run_simulation(sim_name_lu_norton,dpsimpy.DirectLinearSolverImpl.SparseLU,True)
ts_dpsim_sparse_norton = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_norton+".csv")
ts_dpsim_sparse_norton_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_norton+"_RightVector.csv")
ts_dpsim_sparse_norton_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_lu_norton+"_LeftVector.csv")

## Run Dynamic Simulation with KLU with partial refactorization

- Model syngens as Thevenin

In [ ]:
%%capture
sim_name_klu_pf_thevenin = example_name+'_KLU_pf_thevenin'
run_simulation(sim_name_klu_pf_thevenin,dpsimpy.DirectLinearSolverImpl.KLU,False)
ts_dpsim_klu_pf_thevenin = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_thevenin+".csv")
ts_dpsim_klu_pf_thevenin_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_thevenin+"_RightVector.csv")
ts_dpsim_klu_pf_thevenin_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_thevenin+"_LeftVector.csv")

- Model syngens as Norton

In [ ]:
%%capture
sim_name_klu_pf_norton = example_name+'_KLU_pf_norton'
run_simulation(sim_name_klu_pf_norton,dpsimpy.DirectLinearSolverImpl.KLU,True)
ts_dpsim_klu_pf_norton = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_norton+".csv")
ts_dpsim_klu_pf_norton_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_norton+"_RightVector.csv")
ts_dpsim_klu_pf_norton_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_pf_norton+"_LeftVector.csv")

## Run Dynamic Simulation with KLU with complete refactorization

- Model syngens as Thevenin

In [ ]:
%%capture
config = dpsimpy.DirectLinearSolverConfiguration()
config.set_partial_refactorization_method(dpsimpy.partial_refactorization_method.no_partial_refactorization)
sim_name_klu_cf_thevenin = example_name+'_KLU_cf_thevenin'
run_simulation(sim_name_klu_cf_thevenin, dpsimpy.DirectLinearSolverImpl.KLU, False, config)
ts_dpsim_klu_cf_thevenin = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_thevenin+".csv")
ts_dpsim_klu_cf_thevenin_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_thevenin+"_RightVector.csv")
ts_dpsim_klu_cf_thevenin_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_thevenin+"_LeftVector.csv")

- Model syngens as Norton

In [ ]:
%%capture
config = dpsimpy.DirectLinearSolverConfiguration()
config.set_partial_refactorization_method(dpsimpy.partial_refactorization_method.no_partial_refactorization)
sim_name_klu_cf_norton = example_name+'_KLU_cf_norton'
run_simulation(sim_name_klu_cf_norton, dpsimpy.DirectLinearSolverImpl.KLU, True, config)
ts_dpsim_klu_cf_norton = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_norton+".csv")
ts_dpsim_klu_cf_norton_rhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_norton+"_RightVector.csv")
ts_dpsim_klu_cf_norton_lhs = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name_klu_cf_norton+"_LeftVector.csv")

## Plot Comparison of G3 Torque

In [ ]:
# var_name = 'GEN3.Te'
# plt.figure()
# plt.plot(ts_dpsim_sparse_thevenin[var_name].time, ts_dpsim_sparse_thevenin[var_name].values)
# plt.plot(ts_dpsim_klu_pf_thevenin[var_name].time, ts_dpsim_klu_pf_thevenin[var_name].values, linestyle='--')

## Define function for evaluation of relative error

In [ ]:
# Constants
close_to_zero_tolerance = 1e-12

# Helper function relative error (relative to vector1)
def relative_error(vector1: np.ndarray, vector2: np.ndarray) -> (np.ndarray, float, int):
    N = len(vector1)
    rel_error = np.zeros(N, dtype=float)
    for i in range(0, N - 1):
        if abs(vector1[i]) > close_to_zero_tolerance:
            rel_error[i] = abs(vector1[i] - vector2[i]) / abs(vector1[i])
    return rel_error, np.max(rel_error), np.argmax(rel_error)


def evaluation_func(sparse_r_vec, sparse_l_vec, klu_r_vec, klu_l_vec):
    # [RV, LV]
    maxerror = [0.0, 0.0]
    maxerror_timestamp = [0, 0]
    maxerror_entry = ['', '']


    # Relative error in right vector
    print('Maximum relative error in right vector for each entry (when > 0)')
    for entry in sparse_r_vec:
        sparse_values = sparse_r_vec[entry].values
        klu_values = klu_r_vec[entry].values

        relative_error_klu_sparse, max_error_rv, max_error_rv_timestamp = relative_error(sparse_values, klu_values)
        if max_error_rv > maxerror[0]:
            maxerror[0] = max_error_rv
            maxerror_timestamp[0] = max_error_rv_timestamp
            maxerror_entry[0] = entry
        if max_error_rv > close_to_zero_tolerance:
            print('Entry {}: {:.3e} at {}s'.format(entry, max_error_rv, max_error_rv_timestamp / 1000.0))
    print('Maximum relative error in right vector: {} {:.3e} {}s'.format(maxerror_entry[0], maxerror[0], maxerror_timestamp[0] / 1000.0))


    # Relative error in left vector
    print('\nMaximum relative error in left vector for each entry (when > 0)')
    for entry in sparse_l_vec:
        sparse_values = sparse_l_vec[entry].values
        klu_values = klu_l_vec[entry].values

        relative_error_klu_sparse, max_error_lv, max_error_lv_timestamp = relative_error(sparse_values, klu_values)
        if max_error_lv > maxerror[1]:
            maxerror[1] = max_error_lv
            maxerror_timestamp[1] = max_error_lv_timestamp
            maxerror_entry[1] = entry
        if max_error_lv > close_to_zero_tolerance:
            print('Entry {}: {:.3e} at {}s'.format(entry, max_error_lv, max_error_lv_timestamp / 1000.0))
    print('Maximum relative error in left vector: {} {:.3e} {}s'.format(maxerror_entry[1], maxerror[1], maxerror_timestamp[1] / 1000.0))

    _rvlv = ['right', 'left']
    print('\nMaximum relative error in all values: {:.3e} at {}s at entry {} in {} vector'.format(
        max(maxerror),
        maxerror_timestamp[np.argmax(maxerror)] / 1000.0,
        maxerror_entry[np.argmax(maxerror)],
        _rvlv[np.argmax(maxerror)])
         )
    return max(maxerror)

## Calculate maximum relative error between all solver configurations

In [ ]:
max_rel_error = 3e-07

In [ ]:
# KLU partial refactorization / SparseLU, thevenin equivalent
max_error = evaluation_func(
    ts_dpsim_sparse_thevenin_rhs, ts_dpsim_sparse_thevenin_lhs,
    ts_dpsim_klu_pf_thevenin_rhs, ts_dpsim_klu_pf_thevenin_lhs
)

assert(max_error < max_rel_error)

In [ ]:
# KLU partial refactorization / SparseLU, norton equivalent
max_error = evaluation_func(
    ts_dpsim_sparse_norton_rhs, ts_dpsim_sparse_norton_lhs,
    ts_dpsim_klu_pf_norton_rhs, ts_dpsim_klu_pf_norton_lhs
)

assert(max_error < max_rel_error)

In [ ]:
# KLU complete refactorization / SparseLU, thevenin equivalent
max_error = evaluation_func(
    ts_dpsim_sparse_thevenin_rhs, ts_dpsim_sparse_thevenin_lhs,
    ts_dpsim_klu_cf_thevenin_rhs, ts_dpsim_klu_cf_thevenin_lhs
)

assert(max_error < max_rel_error)

In [ ]:
# KLU complete refactorization / SparseLU, norton equivalent
max_error = evaluation_func(
    ts_dpsim_sparse_norton_rhs, ts_dpsim_sparse_norton_lhs,
    ts_dpsim_klu_cf_norton_rhs, ts_dpsim_klu_cf_norton_lhs
)

assert(max_error < max_rel_error)

In [ ]:
# KLU complete refactorization / KLU partial refactorization, thevenin equivalent
max_error = evaluation_func(
    ts_dpsim_klu_pf_thevenin_rhs, ts_dpsim_klu_pf_thevenin_lhs,
    ts_dpsim_klu_cf_thevenin_rhs, ts_dpsim_klu_cf_thevenin_lhs
)

assert(max_error < max_rel_error)

In [ ]:
# KLU complete refactorization / KLU partial refactorization, norton equivalent
max_error = evaluation_func(
    ts_dpsim_klu_pf_norton_rhs, ts_dpsim_klu_pf_norton_lhs,
    ts_dpsim_klu_cf_norton_rhs, ts_dpsim_klu_cf_norton_lhs
)

assert(max_error < max_rel_error)